# Project Outline

#  Etracting values from simulation results

In [69]:
# will take each pdb file and parse it

def file_extract(file):
    """
    Takes a file and extracts the atom type, atom number,
    xposition, ypostion, and zposition and puts them into seperate lists.
    Note that each frame is not seperated yet. 
    
    Inputs:
    file- pdb file, assumed that file is in the same directory
    
    Returns: atomtype, atomnumber, xposition, yposition, zpositions in lists
    
    Example:
    >>> file_extract("example.pdb")
    (['el10'], ['el1'], ['el6'], ['el7'], ['el8'])
    """
    
    file= open(file)

    #creates empty lists for the sparsed file to go into
    atomtype=[]
    atomnumber=[]
    xposition=[]
    yposition=[]
    zposition=[]
    
    for line in file:
        splitline = line.split() #splits each line of file so each goup of text is a string basically omits spaces
        if splitline[0] == "HETATM":
            atomtype.append(splitline[-1])
            atomnumber.append(splitline[1])
            xposition.append(splitline[6])
            yposition.append(splitline[7])
            zposition.append(splitline[8])

    return atomtype, atomnumber, xposition, yposition, zposition  



avgxposition=[]
for atom in range(6000):
    if atomnumber == atom:
        templist=[]
        templist.append(xposition[atom])

 


In [73]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=1)